In [2]:
%matplotlib inline
import pandas as pd
import pylab
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sci
import fastdtw
from scipy.spatial.distance import euclidean

import random
from scipy.cluster.hierarchy import dendrogram, linkage
import scipy.spatial.distance as ssd
from sklearn.ensemble import RandomForestClassifier
import sklearn
from sklearn.neural_network import MLPClassifier


In [28]:
meta=pd.read_csv("/dcs/17/u1403100/Downloads/A2/training_set_metadata.csv")
obs=pd.read_csv("/dcs/17/u1403100/Downloads/A2/training_set.csv")
test_meta=pd.read_csv("/modules/cs342/Assignment2/test_set_metadata.csv")




#test_obs=pd.read_csv("/modules/cs342/Assignment2/test_set.csv")


In [38]:
meta['last_row']=0
previous_id=0
start=0
i=0
read_rows=0
for chunk in pd.read_csv("/dcs/17/u1403100/Downloads/A2/training_set.csv", chunksize=10000):
    for index, row in chunk.iterrows():
        if row['object_id']!=previous_id and start!=0:
            meta.loc[i,'last_row']=index-1+read_rows
            i+=1
        previous_id=row['object_id']
        start=1
    read_rows+=10000

In [42]:
test_meta['last_row']=0
previous_id=0
start=0
i=0
read_rows=0
for chunk in pd.read_csv("/modules/cs342/Assignment2/test_set.csv", chunksize=100000):
    for index, row in chunk.iterrows():
        if row['object_id']!=previous_id and start!=0:
            test_meta.loc[i,'last_row']=index-1+read_rows
            i+=1
        previous_id=row['object_id']
        start=1
    read_rows+=100000
    
    

In [43]:
test_meta.to_csv("/dcs/17/u1403100/Downloads/A2/meta2.csv")

In [5]:
def timeplot(band,id):
    fig = plt.figure()
    ax = plt.axes()
    ax.plot(obs.mjd[(obs.passband==band)&(obs.object_id==id)],obs.flux[(obs.passband==band)&(obs.object_id==id)])

In [ ]:
       """     x=obs.mjd[(obs.passband==2)&(obs.object_id==obs.object_id.unique()[5])]
            y=obs.mjd[(obs.passband==2)&(obs.object_id==obs.object_id.unique()[1])]
            distance, path = fastdtw(x, y, dist=euclidean)
            dist_m[i][j]=distance
            print(distance)
dist_m[5][1]"""

In [ ]:
"""def dtw_m(num):
    dist_m=np.zeros((num,num))
    for i in range(num):
        x=obs.flux[(obs.passband==2)&(obs.object_id==obs.object_id.unique()[i])]
        for j in range(num):
            if i!=j:
                y=obs.flux[(obs.passband==2)&(obs.object_id==obs.object_id.unique()[j])]
                distance, path = fastdtw(x, y, dist=euclidean)
                dist_m[i][j]=distance
    return dist_m
            """


In [ ]:
"""from time import time
t0 = time()
dist_m=dtw_m(20)    
distArray = ssd.squareform(dist_m)

z=linkage(distArray)
labels = sci.cluster.hierarchy.fcluster(z, 3, criterion='maxclust')

t1 = time()
print('function vers1 takes %f' %(t1-t0))"""

In [23]:

obs_test=obs
meta_test=meta
full_test=obs_test.merge(meta_test,on="object_id")
full_x=full_test.drop(["target","distmod"],axis=1)
full_y=full_test["target"]



clf=RandomForestClassifier(n_estimators=80 ,min_samples_leaf=22)

clf.fit(full_x,full_y)





RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=22,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=80, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

function vers1 takes 0.000022


In [19]:

obs_test=obs
meta_test=meta
full_test=obs_test.merge(meta_test,on="object_id")
full_x=full_test.drop(["target","distmod"],axis=1)
full_y=full_test["target"]





mlp = MLPClassifier(max_iter=70,hidden_layer_sizes=(40,60,40),activation='tanh',solver='adam',alpha=0.00005,learning_rate='constant')



mlp.fit(full_x,full_y)


MLPClassifier(activation='tanh', alpha=5e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(40, 60, 40), learning_rate='constant',
       learning_rate_init=0.001, max_iter=70, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [24]:
from time import time
t0 = time()
a=0
for chunk in pd.read_csv("/modules/cs342/Assignment2/test_set.csv", chunksize=100000):
    # process the chunk
    full_test=chunk.merge(test_meta,on="object_id",how='inner')
    full_test2=full_test.drop(["distmod"],axis=1)
    full_test2.hostgal_specz.fillna(full_test2.hostgal_photoz, inplace=True)
    #print(pd.isnull(full_test2).sum() > 0)
    pred=clf.predict_proba(full_test2)
    
    




    dataset = pd.DataFrame({'object_id':full_test2.object_id,
                        
                        'Class_15':pred[:,1],
                        'Class_16':pred[:,2],
                        'Class_42':pred[:,3],
                        'Class_52':pred[:,4],
                        'Class_53':pred[:,5],
                        'Class_62':pred[:,6],
                        'Class_64':pred[:,7],
                        'Class_65':pred[:,8],
                        'Class_67':pred[:,9],
                        'Class_88':pred[:,10],
                        'Class_90':pred[:,11],
                        'Class_92':pred[:,12],
                        'Class_95':pred[:,13]                              
                       })
    dataset['class_99']=0
    dataset.insert(loc=0, column='Class_6', value=pred[:,0])
    output=dataset.groupby('object_id').mean()

    if a==0:
        output.to_csv("/dcs/17/u1403100/Downloads/A2/output.csv")
    else:
        output.to_csv("/dcs/17/u1403100/Downloads/A2/output.csv", mode='a',header=False)
    a=1
t1 = time()
print('function vers1 takes %f' %(t1-t0))

function vers1 takes 10723.289245


In [20]:
a=0
for chunk in pd.read_csv("/modules/cs342/Assignment2/test_set.csv",  chunksize=100000):
    # process the chunk
    full_test=chunk.merge(test_meta,on="object_id",how='inner')
    full_test2=full_test.drop(["distmod"],axis=1)
    full_test2.hostgal_specz.fillna(full_test2.hostgal_photoz, inplace=True)
    pred=mlp.predict_proba(full_test2)
    
    




    dataset = pd.DataFrame({'object_id':full_test2.object_id,
                        
                        'Class_15':pred[:,1],
                        'Class_16':pred[:,2],
                        'Class_42':pred[:,3],
                        'Class_52':pred[:,4],
                        'Class_53':pred[:,5],
                        'Class_62':pred[:,6],
                        'Class_64':pred[:,7],
                        'Class_65':pred[:,8],
                        'Class_67':pred[:,9],
                        'Class_88':pred[:,10],
                        'Class_90':pred[:,11],
                        'Class_92':pred[:,12],
                        'Class_95':pred[:,13]                       
                       })
    dataset['class_99']=0
    dataset.insert(loc=0, column='Class_6', value=pred[:,0])
    output=dataset.groupby('object_id').mean()

    if a==0:
        output.to_csv("/dcs/17/u1403100/Downloads/A2/output2.csv")
    else:
        output.to_csv("/dcs/17/u1403100/Downloads/A2/output2.csv", mode='a',header=False)
    a=1

In [25]:
output=pd.read_csv("/dcs/17/u1403100/Downloads/A2/output.csv")
output2=output.groupby('object_id').mean()
output2.to_csv("//var/tmp/kdecache-u1403100/Q21.csv")

In [22]:
output2.shape

(3492890, 15)

In [16]:
"""output=pd.read_csv("//var/tmp/kdecache-u1403100/output.csv")
output['class_53']=0
output2=output.groupby('object_id').mean()
output2.to_csv("/dcs/17/u1403100/Downloads/A2/Q21.csv")"""

(3492890, 11)

In [ ]:
"""from sklearn.model_selection import KFold
df2 = pd.DataFrame(np.zeros((10,10)) )  
t0 = time()
test=meta.object_id.sample(n=150)
obs_test=obs[obs.object_id.isin(list(test))]
meta_test=meta[meta.object_id.isin(list(test))]
full_test=obs_test.merge(meta_test,on="object_id")
full_x=full_test.drop(["target","distmod"],axis=1)
full_y=full_test["target"]
for i in range(1):
    for k in range(10):
        result=0
        
        
        kf = KFold(n_splits=6)
        for train_index, test_index in kf.split(full_x):
           
            x_train, x_test = full_x.iloc[train_index], full_x.iloc[test_index]
            y_train, y_test = full_y.iloc[train_index], full_y.iloc[test_index]
           

            clf = RandomForestClassifier(n_estimators=60 ,min_samples_leaf=(k*5)+1)
            
            
            clf.fit(x_train,y_train)
            
            #ans=clf.predict(x_test)
            
            result+=clf.score(x_test,y_test)
            #print(sklearn.metrics.r2_score(y_test,ans))
            #print(neigh.score(X_test,y_test),"  ",end="")
        # print("pa",i)
        # print(k_val)
        #print(np.average(result))
        #print(result)
        df2.ix[i,k]=result/(6)
    
    print("BREAK")
print(df2)

t1 = time()
print('function vers1 takes %f' %(t1-t0))"""

In [ ]:
"""from sklearn.model_selection import KFold
df2 = pd.DataFrame(np.zeros((10,10)) )  
t0 = time()

for i in range(1):
    for k in range(10):
        result=0
        
        
        
        for j in range(6):
            test_index=meta.object_id.sample(n=300)
            train_index=meta.object_id.sample(n=100)
            
            obs_test=obs[obs.object_id.isin(list(test_index))]
            meta_test=meta[meta.object_id.isin(list(test_index))]
            full_test=obs_test.merge(meta_test,on="object_id")
            full_x=full_test.drop(["target","distmod"],axis=1)
            full_y=full_test["target"]
            
            obs_train=obs[obs.object_id.isin(list(train_index))]
            meta_train=meta[meta.object_id.isin(list(train_index))]
            full_train=obs_train.merge(meta_train,on="object_id")
            full_x_train=full_train.drop(["target","distmod"],axis=1)
            full_y_train=full_train["target"]
            clf = RandomForestClassifier(n_estimators=70 ,min_samples_leaf=(k*8)+1)
            
            
            clf.fit(full_x,full_y)
            
            #ans=clf.predict(x_test)
            
            result+=clf.score(full_x_train,full_y_train)
            #print(sklearn.metrics.r2_score(y_test,ans))
            #print(neigh.score(X_test,y_test),"  ",end="")
        # print("pa",i)
        # print(k_val)
        #print(np.average(result))
        #print(result)
        df2.ix[i,k]=result/(6)
    
    print("BREAK")
print(df2)

t1 = time()
print('function vers1 takes %f' %(t1-t0))"""